In [1]:
from contextlib import contextmanager
from datetime import datetime
from glob import glob
from time import time, sleep
import feather
import logging
import numpy as np
import os
import pandas as pd
import platform
import sys

In [2]:
def start(fname):
    global st_time
    st_time = time()
    print("="*80)
    print("DATE: {}".format(datetime.now()))
    print("FILE: {}".format(fname))
    print("PID: {}".format(os.getpid()))
    print("HOST: {}".format(platform.node()))
    print("ENV: {}".format(platform.platform()))
    print("="*80)


def end():
    print("="*80)
    print("ELAPSED: {:.2f} (sec)".format(elapsed_time()))
    print("="*80)


def elapsed_time():
    return (time() - st_time)


def reset_time():
    global st_time
    st_time = time()

In [3]:
def to_feather(df, path="./"):
    df.reset_index(inplace=True, drop=True)
    os.makedirs(path, exist_ok=True)
    print(f"write to {path}")
    for c in df.columns:
        path_file = os.path.join(path, f"{c}.f")
        if not glob(path_file):
            df[[c]].to_feather(path_file)
        else:
            print(f"WARNIG: {path_file} is exists!")
            sys.exit()

            
def read_feather(path="./", col=None):
    if col is None:
        path_file = os.path.join(path, "*.f")
        print(f"read {path_file}")
        df = pd.concat([ feather.read_dataframe(f) for f in sorted(glob(path_file)) ], axis=1)
    else:
        path_file = os.path.join(path, col)
        if not glob(path_file):
            print(f"read {path_file}")
            df = feather.read_dataframe(path_file)
        else:
            print(f"WARNIG: {path_file} is exists!")
            sys.exit()
    return df

In [4]:
def to_pickles(df, path="./"):
    df.reset_index(inplace=True, drop=True)
    os.makedirs(path, exist_ok=True)
    print(f"write to {path}")
    for c in df.columns:
        path_file = os.path.join(path, f"{c}.f")
        if not glob(path_file):
            df[[c]].to_pickle(path_file)
        else:
            print(f"WARNIG: {path_file} is exists!")
            sys.exit()


def read_pickles(path="./", col=None):
    if col is None:
        path_file = os.path.join(path, "*.f")
        print(f"read {path_file}")
        df = pd.concat([ pd.read_pickle(f) for f in sorted(glob(path_file)) ], axis=1)
    else:
        path_file = os.path.join(path, col)
        if not glob(path_file):
            print(f"read {path_file}")
            df = pd.read_pickle(path_file)
        else:
            print(f"WARNIG: {path_file} is exists!")
            sys.exit()
    return df

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
@contextmanager
def timer(name, logger=None, level=logging.DEBUG):
    print_ = print if logger is None else lambda msg: logger.log(level, msg)
    t0 = time()
    print_(f'[{name}] start')
    yield
    print_(f'[{name}] done in {time() - t0:.0f} s')

In [7]:
def get_dummies(df):
    col = df.select_dtypes('O').columns.tolist()
    nunique = df[col].nunique()
    col_binary = nunique[nunique==2].index.tolist()
    [col.remove(c) for c in col_binary]
    df = pd.get_dummies(df, columns=col)
    df = pd.get_dummies(df, columns=col_binary, drop_first=True)
    return df

In [8]:
def _target_mean_encoding(df, src_col, tgt_col, prfx, conn):
    """
    target mean encoding
    target が数値のときに利用できる
    """
    target_mean = df.groupby(src_col)[tgt_col].mean().astype("float16")
    df = df[src_col].map(target_mean).copy()
    df.name = prfx+str(src_col)+conn+str(tgt_col)+"_target_mean"
    return df


def target_mean_encoding(df, tgt_col, prfx="@", conn="_"):
    """
    target mean encoding
    すべてのカテゴリ変数に target mean encoding
    """
    src_col = df.select_dtypes("O").columns.tolist()
    if tgt_col in src_col: src_col.remove(tgt_col)
    df = pd.concat([ _target_mean_encoding(df, c, tgt_col, prfx, conn) for c in src_col ], axis=1)
    return df


def _bin_counting(df, src_col, tgt_col, prfx, conn):
    """
    bin counting
    target がカテゴリのときに利用できる
    """
    a = df[src_col]
    b = df[tgt_col]
    cross_df         = pd.crosstab(a, b, rownames=["a"], colnames=["b"], normalize="index")
    labels           = [prfx+str(src_col)+conn+str(tgt_col)+conn+str(col)+"_bin_counting" for col in cross_df.columns]
    cross_df.columns = labels
    df  = pd.merge(df, cross_df, left_on=src_col, right_index=True, how="left")
    return df[labels]


def bin_counting(df, tgt_col, prfx="@", conn="_"):
    """
    bin counting
    すべてのカテゴリ変数と tgt_col に bin counting
    """
    src_col = df.select_dtypes('O').columns.tolist()
    if tgt_col in src_col: src_col.remove(tgt_col)
    df = pd.concat([ _bin_counting(df, c, tgt_col, prfx, conn) for c in src_col ], axis=1)
    return df

In [9]:
# バックオフ：少数派カテゴリ値を抽出
def _backoff_cand(df, cutoff=0.01):
    col = df.select_dtypes('O').columns.tolist()
    cand = {c: [idx for idx, val in df[c].value_counts(normalize=True, dropna=False).iteritems() if val < cutoff] for c in col}
    cand = {key: val for key, val in cand.items() if val!=[]} # 空リストを削除
    return cand


# バックオフ：少数派カテゴリ値をダミーで置き換え
def _backoff_replace(df, cand, repl, prfx, rep):
    _df = pd.DataFrame({})
    col = []
    for key, val in cand.items():
        tmp = df[key]
        col_name = prfx+str(key)
        for idx in val:
            tmp = tmp.replace(idx, rep)
        _df[col_name] = tmp
        col.append(col_name)
    return _df


def backoff(df, prfx="@", rep="@backoff", repl=False):
    cand = _backoff_cand(df)
    _df = _backoff_replace(df, cand, repl, prfx, rep)
    if repl:
        for key, val in cand.items():
            df[key] = _df[prfx+str(key)]
        return df
    else:
        return pd.concat([df, _df], axis=1)

In [11]:
def send_line_notification(message, png=None):
    line_notify_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    payload = {'message': message}
    headers = {'Authorization': 'Bearer ' + line_notify_token}
    if png is None:
        requests.post(line_notify_api, data=payload, headers=headers)
    else:
        files = {"imageFile": open(png, "rb")}
        requests.post(line_notify_api, data=payload, headers=headers, files=files)
    print(message)

In [10]:
start("hoge")

DATE: 2019-04-02 09:56:16.313633
FILE: hoge
PID: 7572
HOST: Y-project-11
ENV: Windows-10-10.0.17134-SP0


In [11]:
with timer("read"):
    file_name = "../../python_codes/data/mushroom.csv"
    df = pd.read_csv(file_name, header=None, index_col=None, sep=',')
    # df = pd.DataFrame({"a":[1,2,3,4,5], "b":[6,7,8,9,10], "c":["a","b","c","d","e"]})
    # df

[read] start
[read] done in 0 s


In [12]:
# to_feather(df, "new_folder00")
# df00 = read_feather("new_folder00")
# df00

In [13]:
# to_pickles(df, "new_folder01")
# df01 = read_pickles("new_folder01")
# df01

In [14]:
with timer("reduce mem usage"):
    df = reduce_mem_usage(df)

[reduce mem usage] start
Mem. usage decreased to  1.43 Mb (0.0% reduction)
[reduce mem usage] done in 0 s


In [15]:
with timer("reduce"):
    get_dummies(df)

[reduce] start
[reduce] done in 0 s


In [16]:
with timer("timer"):
    for _ in range(10):
        sleep(0.1)

[timer] start
[timer] done in 1 s


In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [18]:
# a = {
#     "cat1": ["a", "b", "a", "a", "c", "c", "c"],
#     "cat2": ["A", "A", "B", "B", "B", "C", "C"],
#     "numeric":  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
#     "label":    [1, 1, 1, 2, 2, 3, 3]
# }
# df = pd.DataFrame(a)

with timer("proprocessing"):
    # print(get_dummies(df).head())
    # print(bin_counting(df, 0).head())
    # print(target_mean_encoding(df, 0).head())
    print(backoff(df, repl=True).head(50))

[proprocessing] start
   0         1  2  3  4  5  6  7  8  9   ... 13 14 15 16 17 18 19 20 21 22
0   p         x  s  n  t  p  f  c  n  k  ...  s  w  w  p  w  o  p  k  s  u
1   e         x  s  y  t  a  f  c  b  k  ...  s  w  w  p  w  o  p  n  n  g
2   e         b  s  w  t  l  f  c  b  n  ...  s  w  w  p  w  o  p  n  n  m
3   p         x  y  w  t  p  f  c  n  n  ...  s  w  w  p  w  o  p  k  s  u
4   e         x  s  g  f  n  f  w  b  k  ...  s  w  w  p  w  o  e  n  a  g
5   e         x  y  y  t  a  f  c  b  n  ...  s  w  w  p  w  o  p  k  n  g
6   e         b  s  w  t  a  f  c  b  g  ...  s  w  w  p  w  o  p  k  n  m
7   e         b  y  w  t  l  f  c  b  n  ...  s  w  w  p  w  o  p  n  s  m
8   p         x  y  w  t  p  f  c  n  p  ...  s  w  w  p  w  o  p  k  v  g
9   e         b  s  y  t  a  f  c  b  g  ...  s  w  w  p  w  o  p  k  s  m
10  e         x  y  y  t  l  f  c  b  g  ...  s  w  w  p  w  o  p  n  n  g
11  e         x  y  y  t  a  f  c  b  n  ...  s  w  w  p  w  o  p  k  s  m
12 

In [19]:
end()

ELAPSED: 1.28 (sec)
